In [1]:
data = sc.textFile('/home/dsc/Data/spark/sales_segments.csv.gz')

In [2]:
data.first()

u'rloc^departure_date_seg^booking_date^freq_flyer_flag^creator_office_id^creator_iata_number^creator_gds_code^creator_city_code^creator_lat^creator_lon^creator_country_code^creator_continent_code^creator_channel_code^booking_status_code^cabin_code^booking_class_code^pos_office_id^pos_iata_number^pos_gds_code^pos_city_code^pos_lat^pos_lon^pos_country_code^pos_continent_code^pos_channel_code^board_point_seg^board_lat^board_lon^board_country_code^board_continent_code^off_point_seg^off_lat^off_lon^off_country_code^off_continent_code^num_legs^distance_seg^route^network^airline_code^flight_number^flight_type^revenue_source^estimated_flag^usage_board_point_seg^usage_off_point_seg^usage_flight_number^usage_date^non_currency_type^bookings_seg^rpk_seg^revenue_amount_seg^currency_code^fuel_surcharge_amount_seg^fuel_surcharge_currency_code^emd_count^emd_amount^emd_currency_code^total_outliers'

In [3]:
data.map(lambda l: l.split('^')[51]).take(3)

[u'revenue_amount_seg', u'33.56', u'306.00']

In [ ]:
#eliminamos el header

In [4]:
header = data.first()

In [5]:
lines = data.filter(lambda line: line != header)

In [6]:
lines.first()

u'59GS9Y^2013-12-21^2013-09-05^false^RUNHS3103^66200201^1A^RUN^-20.90083^55.53715^RE^Africa^Travel Agent^HK^Y^N^RUNHS3103^66200201^1A^RUN^-20.90083^55.53715^RE^Africa^Travel Agent^ZYR^50.8348^4.33653^BE^Europe^CDG^49.01278^2.55^FR^Europe^0^239.573363253866^CDG-ZYR^Europe^UU^5901^I^LIFT^false^ZYR^CDG^5901^2013-12-21^^1^239^33.56^EUR^7.30^EUR^0^0.00^^7.3'

In [ ]:
#sacamos los campos que nos interesan

In [13]:
fields = ['bookings_seg', 'revenue_amount_seg', 'fuel_surcharge_amount_seg']

In [14]:
[header.split('^').index(field) for field in fields]

[49, 51, 53]

In [15]:
def average_fare_tax(line):
    elems = line.split('^')
    bookings = float(elems[49])
    rev = float(elems[51])
    tax = float(elems[53])
    
    av_rev = rev / bookings
    av_tax = tax / bookings
    
    return (av_rev, av_tax)

In [16]:
average_fare_tax(lines.first())

(33.56, 7.3)

In [17]:
simple = lines.map(average_fare_tax)

In [19]:
simple.take(3)

[(33.56, 7.3), (61.2, 25.5), (71.86842105263158, 12.31578947368421)]

In [ ]:
#Kmeans clustering

In [21]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [22]:
clusters = KMeans.train(simple, 10)

In [23]:
clusters.save(sc,'clusters')

In [24]:
clusters.centers

[array([ 109.0339497 ,   63.70834282]),
 array([  8.77500428e+05,   3.03020000e+01]),
 array([  3.46736736e+05,   1.02778125e+02]),
 array([  1.12648224e+05,   2.12891489e+01]),
 array([  6.15206951e+05,   2.97312500e+01]),
 array([  1.39215574e+06,   3.27400000e+01]),
 array([ 17017.06404697,     87.99088939]),
 array([  5.02409789e+04,   3.25942361e+01]),
 array([ 429.22751871,  184.7775527 ]),
 array([ 1516.91592787,   209.38205557])]

In [27]:
clusters.predict((33.56, 7.3))

0

In [30]:
bycluster = simple.map(lambda point: (clusters.predict(point), point))

In [31]:
bycluster.take(3)

[(0, (33.56, 7.3)),
 (0, (61.2, 25.5)),
 (0, (71.86842105263158, 12.31578947368421))]

In [ ]:
#calcular media y sigma de cada punto

In [32]:
bycluster.cache()

PythonRDD[120] at RDD at PythonRDD.scala:43

In [ ]:
#means = bycluster.reduceByKey(calc_avg_reduce)

In [33]:
bycluster_withones = simple.map(lambda (x,y): (clusters.predict((x,y)), (x,y,1)))

In [34]:
bycluster_withones.take(3)

[(0, (33.56, 7.3, 1)),
 (0, (61.2, 25.5, 1)),
 (0, (71.86842105263158, 12.31578947368421, 1))]

In [39]:
bycluster_withones.cache()

PythonRDD[132] at RDD at PythonRDD.scala:43

In [35]:
def calc_avg_reduce(acc, p):
    (rev_p, tax_p, c) = p
    (rev_ac, tax_ac, c_acc) = acc
    
    result = (rev_p + rev_ac, tax_p + tax_ac, c + c_acc)
    
    return result

In [37]:
avs = bycluster_withones.reduceByKey(calc_avg_reduce)

In [38]:
avs.take(3)

[(0, (118179717.4017657, 69052198.6209513, 1083880)),
 (1, (8775004.28, 303.02, 10)),
 (2, (5547787.77, 1644.45, 16))]

In [ ]:
#hacemos un join para asignar a cada punto la media de su grupo.

In [44]:
def medias(clust):
    rev_t, tax_t, count = clust
    return (rev_t / count, tax_t / count, count)

In [45]:
avs.mapValues(medias).take(3)

[(0, (109.03394970085776, 63.70834282480652, 1083880)),
 (1, (877500.428, 30.302, 10)),
 (2, (346736.735625, 102.778125, 16))]

In [46]:
averages = avs.mapValues(medias)

In [ ]:
#Esto esta mal

In [73]:
#simple.join(averages).take(3)

In [ ]:
#Así mejor

In [47]:
bycluster_withones.join(averages).take(3)

[(0, ((33.56, 7.3, 1), (109.03394970085776, 63.70834282480652, 1083880))),
 (0, ((61.2, 25.5, 1), (109.03394970085776, 63.70834282480652, 1083880))),
 (0,
  ((71.86842105263158, 12.31578947368421, 1),
   (109.03394970085776, 63.70834282480652, 1083880)))]

In [48]:
def sq_diffs(p_with_cluster_mean):
    
    ((rev, tax, n),(av_rev, av_tax, n_cl)) = p_with_cluster_mean
    
    rev_sqdiff = (rev - av_rev) ** 2
    tax_sqdiff = (tax - av_tax) ** 2
    
    return (rev_sqdiff, tax_sqdiff, 1)

In [49]:
points_w_means_sqdiffs = bycluster_withones.join(averages).mapValues(sq_diffs)

In [50]:
points_w_means_sqdiffs.take(3)

[(0, (5696.317083447607, 3181.901140240902, 1)),
 (0, (2288.08674398419, 1459.877461417944, 1)),
 (0, (1381.2765197021213, 2641.1945399479528, 1))]

In [53]:
total_sqdiffs = points_w_means_sqdiffs.reduceByKey(calc_avg_reduce)

In [54]:
total_sqdiffs.take(3)

[(0, (5876003118.069857, 5449599804.500944, 1083880)),
 (8, (9036820149.117517, 1235284075.3588262, 354044)),
 (2, (41281701393.02079, 191010.53404375003, 16))]

In [55]:
from math import sqrt
def stdevs(sqdiffs_percluster):
    (sqd_rev_total, sqd_tax_total, n_cl) = sqdiffs_percluster
    
    std_rev = sqrt(sqd_rev_total / n_cl)
    std_tax = sqrt(sqd_tax_total / n_cl)
    
    return (std_rev, std_tax)

In [57]:
total_sqdiffs.mapValues(stdevs).take(11)

[(0, (73.62925521403366, 70.90742332561464)),
 (8, (159.76410106484028, 59.068344890455705)),
 (2, (50794.74714046522, 109.26187980139449)),
 (4, (49736.26150524646, 1.442501624782446)),
 (6, (5850.329017210933, 84.6638736432238)),
 (1, (47658.199670890375, 1.7160000000000006)),
 (3, (21865.59108696715, 65.48987338739448)),
 (9, (581.6406112916662, 62.88919766934884)),
 (5, (0.0, 0.0)),
 (7, (9752.495674914668, 38.94671868675312))]

In [58]:
stds_clusters = total_sqdiffs.mapValues(stdevs)

In [59]:
cluster_stats = averages.join(stds_clusters)

In [62]:
cluster_stats.take(3)

[(0,
  ((109.03394970085776, 63.70834282480652, 1083880),
   (73.62925521403366, 70.90742332561464))),
 (9,
  ((1516.9159278705047, 209.38205557371586, 20977),
   (581.6406112916662, 62.88919766934884))),
 (3,
  ((112648.22446808493, 21.289148936170214, 94),
   (21865.59108696715, 65.48987338739448)))]

In [65]:
full = bycluster.join(cluster_stats)

In [66]:
full.take(3)

[(0,
  ((33.56, 7.3),
   ((109.03394970085776, 63.70834282480652, 1083880),
    (73.62925521403366, 70.90742332561464)))),
 (0,
  ((61.2, 25.5),
   ((109.03394970085776, 63.70834282480652, 1083880),
    (73.62925521403366, 70.90742332561464)))),
 (0,
  ((71.86842105263158, 12.31578947368421),
   ((109.03394970085776, 63.70834282480652, 1083880),
    (73.62925521403366, 70.90742332561464))))]

In [75]:
def zscore(p_w_stats):
    
    (p, stats) = p_w_stats
    rev, tax = p
    means, stdevs = stats
    (rev_m, tax_m, _) = means
    (rev_std, tax_std) = stdevs
    
    
    if rev_std == 0:
        return rev, tax, 3000, True 
        
    zscore_rev = (rev - rev_m) / rev_std
    zscore_tax = (tax - rev_m) / tax_std
    
    zscore = sqrt(zscore_rev ** 2 + zscore_tax ** 2)
    outlier = zscore > 3
    
    return rev, tax, zscore, outlier

In [70]:
zscore(full.first()[1])

(33.56, 7.3, 1.7632990805720967, False)

In [76]:
outliers = full.mapValues(zscore)

In [72]:
outliers.take(3)

[(0, (33.56, 7.3, 1.7632990805720967, False)),
 (0, (61.2, 25.5, 1.34532815723209, False)),
 (0, (71.86842105263158, 12.31578947368421, 1.454407516454938, False))]

In [77]:
outliers.saveAsTextFile('outliers')